## Part IV.16: Experiments with Transformer Model Architecture

In this section, experiments and tuning exercises will be conducted using Transformer foundational model architecture.  This will include the following:

1.  Experiments with different depths of Transformer model architectures (number of encoder layers, etc.)
2.  Hyperparameter tuning/optimization